# Problema da Mochila Booleana

Seja o conjunto de $n$ itens, cada um com uma descrição de peso $P = (p_{0}, \dots, p_{n - 1})$ e uma descrição de benefício $V = (v_{0}, \dots, v_{n - 1})$. Sendo a capacidade da mochila dada por $W$. O objetivo é maximizar o somatório $\sum_{i = 0}^{n - 1}{V[i] * X[i]}$, sendo $X$ o conjunto contendo os itens.

#### Links:
KNAPSACK ALLOCATION OF MULTIPLE RESOURCES 
IN BENEFIT-COST ANALYSIS 
BY WAY OF THE ANALYTIC HIERARCHY PROCESS 

https://lume.ufrgs.br/handle/10183/66091
https://link.springer.com/chapter/10.1007/3-540-59408-6_44?noAccess=true
https://www.sciencedirect.com/science/article/pii/S1572528606000909

#### Limpeza da Pasta de dados

In [1]:
!rm -r sample_data/

In [2]:
!mkdir sample_data

#### Instalação de Pacotes adcionais

In [3]:
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
! pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [6]:
! cp kaggle.json ~/.kaggle/

In [7]:
! chmod 600 ~/.kaggle/kaggle.json

#### Pacotes usados

In [8]:
import math

In [9]:
import wget

In [10]:
import zipfile

In [11]:
import numpy as np

In [12]:
import pandas as pd

In [13]:
from time import time

In [14]:
from functools import lru_cache

### Extrais dados

In [15]:
! kaggle datasets download raullima/problema-knapsack

problema-knapsack.zip: Skipping, found more recently modified local copy (use --force to force download)


In [16]:
def extract_dataset_zip():
  with zipfile.ZipFile('/content/problema-knapsack.zip', mode='r') as dataZip:
    dataZip.extractall(path='/content/sample_data')

In [17]:
extract_dataset_zip()

In [18]:
!ls sample_data/ | egrep '_[1-3]$' > input_name_file.txt

In [19]:
!ls sample_data/ | egrep '_[1-3]o$' > output_name_file.txt

In [20]:
!ls sample_data/ | egrep '_[1-3]a$' > answer_name_file.txt

In [21]:
name_file:pd.DataFrame = pd.DataFrame(
    pd.read_csv(
        'input_name_file.txt',
        sep=' ',
        header=None
    )
)

In [22]:
output_file:pd.DataFrame = pd.DataFrame(
    pd.read_csv(
        'output_name_file.txt',
        sep=' ',
        header=None
    )
)

In [23]:
answer_file:pd.DataFrame = pd.DataFrame(
    pd.read_csv(
        'answer_name_file.txt',
        sep=' ',
        header=None
    )
)

In [24]:
name_file.rename(columns={name_file.columns[0]: 'input_file'})

,input_file
0,knapPI_1_10000_1000_1
1,knapPI_1_1000_1000_1
2,knapPI_1_100_1000_1
3,knapPI_1_2000_1000_1
4,knapPI_1_200_1000_1
5,knapPI_1_5000_1000_1
6,knapPI_1_500_1000_1
7,knapPI_2_10000_1000_1
8,knapPI_2_1000_1000_1
9,knapPI_2_100_1000_1


In [25]:
output_file.rename(columns={output_file.columns[0]: 'output_file'})

,output_file
0,knapPI_1_10000_1000_1o
1,knapPI_1_1000_1000_1o
2,knapPI_1_100_1000_1o
3,knapPI_1_2000_1000_1o
4,knapPI_1_200_1000_1o
5,knapPI_1_5000_1000_1o
6,knapPI_1_500_1000_1o
7,knapPI_2_10000_1000_1o
8,knapPI_2_1000_1000_1o
9,knapPI_2_100_1000_1o


In [26]:
answer_file.rename(columns={answer_file.columns[0]: 'answer_file'})

,answer_file
0,knapPI_1_10000_1000_1a
1,knapPI_1_1000_1000_1a
2,knapPI_1_100_1000_1a
3,knapPI_1_2000_1000_1a
4,knapPI_1_200_1000_1a
5,knapPI_1_5000_1000_1a
6,knapPI_1_500_1000_1a
7,knapPI_2_10000_1000_1a
8,knapPI_2_1000_1000_1a
9,knapPI_2_100_1000_1a


In [27]:
name_file = name_file.join(output_file, lsuffix='input_file', rsuffix='output_file')

In [28]:
name_file = name_file.join(answer_file, lsuffix='input_file', rsuffix='answer_file')

In [29]:
name_file = name_file.rename(columns={
    name_file.columns[0]: 'input_file', 
    name_file.columns[1]: 'output_file',
    name_file.columns[2]: 'answer_file'
    }
)

### Carregameto dos dados

In [30]:
def load_data(name:str, sort_by:str) -> tuple:
  data = pd.DataFrame(
        pd.read_csv(
        '/content/sample_data/'+ name,
        sep=' ',
        header=None
    )
  )
  data = data.reset_index()
  data = data.rename(columns={data.columns[0]: 'indice', data.columns[1]: "value", data.columns[2]: 'weight'})
  data = data.sort_values(by=sort_by)
  return data

In [39]:
def load_data_weight(name:str) -> int:
  data = pd.DataFrame(
        pd.read_csv(
        '/content/sample_data/' + name,
        sep=' ',
    )
  )

  weight:int = int(data.columns[1])
  return weight

In [32]:
def load_data_cost(name:str) -> int:
  data:int = 0
  with open('/content/sample_data/' + name, mode='r') as file:
    data = int(file.readline())
  return data

In [33]:
def load_data_answer(name:str) -> np.array:
  data:np.array = np.array([])
  data = np.loadtxt('/content/sample_data/' + name, delimiter=' ')
  
  return data

In [91]:
def load_data_benefit_by_cust(name:str) -> pd.DataFrame:
  dataframe:pd.DataFrame = pd.DataFrame(
      pd.read_csv(
          '/content/sample_data/'+name,
          sep=' ',
          header=None
      )
  )

  dataframe = dataframe.reset_index()
  dataframe = dataframe.rename(
      columns={
          dataframe.columns[0]: 'indice', 
          dataframe.columns[1]: "value", 
          dataframe.columns[2]: 'weight'
          }
  )


  
  values:np.array = dataframe.value
  weights:np.array = dataframe.weight
  benefit_by_cust_array:np.array = np.divide(values, weights)
  benefit_by_cust_array = np.trunc(benefit_by_cust_array)

  dataframe['benefit_by_cust'] = benefit_by_cust_array
  dataframe = dataframe.sort_values(by='benefit_by_cust')
  return dataframe

In [40]:
def load_datas_1(names:pd.DataFrame, lista_data:list) -> list:
  for i in names.iterrows():
    lista_data.append(
      [
        load_data(i[1].input_file, 'weight'),
        load_data_weight(i[1].input_file),
        load_data_cost(i[1].output_file),
        load_data_answer(i[1].answer_file)
      ]
    )
  
  return lista_data

In [92]:
def load_datas_2(names:pd.DataFrame, lista_data:list) -> list:
  for _, i in names.iterrows():
    lista_data.append(
      [
        load_data_benefit_by_cust(i.input_file),
        load_data_weight(i.input_file),
        load_data_cost(i.output_file),
        load_data_answer(i.answer_file)
      ]
    )
  
  return lista_data

### Itens usados

In [88]:
lista_data_file1:list  = []
lista_data_file2:list  = []

In [89]:
lista_data_file1 = load_datas_1(name_file, lista_data_file1)

In [93]:
lista_data_file2 = load_datas_2(name_file, lista_data_file2)

In [ ]:
knapSack:pd.DataFrame = pd.DataFrame()

## Abordagem Gulosa

### Insere os itens de menor peso, procurando maximizar o número de itens inseridos

In [44]:
def by_minimu_weigths(weight_knap_sack:int, 
                  knap_sack:pd.DataFrame) -> pd.DataFrame:
  insert_item:int = 0
  iten_weight:int = 0
  min_weight_knap_sack:int = 0
  set_solution:pd.DataFrame = pd.DataFrame({
      'indice': [],
      'value': [],
      'weight': []
  })

  for _, item in knap_sack.iterrows(): 
    iten_weight = item.weight
    if iten_weight < weight_knap_sack - insert_item:
      set_solution = set_solution.append({
        'indice': item.indice,
        'value': item.value,
        'weight': item.weight
      }, ignore_index=True)
      weight_knap_sack = weight_knap_sack - insert_item
      insert_item = item.weight

  return set_solution

### Insere os itens de melhor relação de benefício/custo

In [47]:
def by_benefit_cust(weight_knap_sack:int, benefit_knap_sack:int,
                 knap_sack:pd.DataFrame) -> pd.DataFrame:
  weight_after:int = 0
  iten_benefit_by_cust:int = 0
  benefit_by_cust_after:int = 0
  max_benefit_by_cust_knap_sack:int = 0
  set_solution:pd.DataFrame = pd.DataFrame({
      'indice': [],
      'value': [],   
      'weight': []
  })

  max_benefit_by_cust_knap_sack = math.trunc(benefit_knap_sack
                                             //weight_knap_sack)

  for _, item in knap_sack.iterrows():
    iten_benefit_by_cust = item.benefit_by_cust
    if iten_benefit_by_cust < max_benefit_by_cust_knap_sack - benefit_by_cust_after \
     and weight_knap_sack - weight_after > weight_after:
      set_solution = set_solution.append({
        'indice': item.indice,
        'value': item.value,   
        'weight': item.weight
      }, ignore_index=True)
      weight_knap_sack = weight_knap_sack - weight_after
      max_benefit_by_cust_knap_sack -= benefit_by_cust_after
      benefit_by_cust_after = iten_benefit_by_cust
      weight_after = item.weight

  return set_solution

## Abordagem Recursiva

In [48]:
class Memoize:
  __memo:dict

  def __init__(self, fn):
    self.fn = fn
    self.__memo = {}
  
  def __call__(self, *args):
    if args not in self.__memo:
      self.__memo[args] = self.fn(*args)
    return self.__memo[args]

In [49]:
#@lru_cache(maxsize=None)
@Memoize
def problem_knap_sack(index:int, weightBagPack:int, values:tuple, weights:tuple) -> int:
  if index == 0:
    return 0
  if weightBagPack == 0:
    return 0
  if weights[index - 1] > weightBagPack:
    return problem_knap_sack(
            index - 1,
            weightBagPack,
            values,
            weights
        )
  return max(
      problem_knap_sack(
          index - 1,
          weightBagPack,
          values,
          weights
      ),
      values[index - 1] + 
             problem_knap_sack(
                 index - 1,
                 weightBagPack - weights[index - 1],
                 values,
                 weights
             )
  )

In [204]:
%time problem_knap_sack(df.shape[0], W, tuple(df.value.tolist()), tuple(df.weight.tolist()))

CPU times: user 92.9 ms, sys: 0 ns, total: 92.9 ms
Wall time: 95.3 ms


9147

## Abordagem Dinamica

### Abordagem Dinamica

In [50]:
class knap_sack_memoize:
  __memo:list

  def __init__(self, number_itens:int, weight_knap_sack:int):
    self.__memo = [
                   [0 for j in range(0, weight_knap_sack)] 
                    for i in range(0, number_itens)
                   ]

  
  def dinamic_knap_sack(self, number_itens:int, weight_knap_sack:int, 
                        knap_sack:pd.DataFrame) -> pd.DataFrame:
    for i in range(1, number_itens ):
      for j in range(1, weight_knap_sack):
        if(knap_sack.weight.get(i - 1) > j):
          self.__memo[i][j] = self.__memo[i - 1][j]
        else:
          self.__memo[i][j] = max(
              self.__memo[i - 1][j], 
              self.__memo[i - 1][j - knap_sack.weight.get(i - 1)] +
              knap_sack.value.get(i - 1)
              )
    
    return pd.DataFrame(self.__memo)

## Execução dos Algoritmos

#### Execução usando o Método Guloso para Valores Minimos

In [72]:
def execute_by_epoch_g_m(epoch:int, data_op:list) -> tuple:
  start_time:float = 0.0
  p_answer:pd.DataFrame = pd.DataFrame({})
  time_execute_by_greedy_algorithm:list = []
  predict_algorithm:list = []
  for _ in range(epoch):
    for data in data_op:
      start_time = time()
      p_answer = by_minimu_weigths(data[1], data[0])
      time_execute_by_greedy_algorithm.append(
          time() - start_time
          )
      predict_algorithm.append(
          p_answer
      )

  return time_execute_by_greedy_algorithm, predict_algorithm

In [74]:
pgm:tuple = execute_by_epoch_g_m(10, lista_data_file1)

In [77]:
pgm

([2.231452703475952,
  0.21205472946166992,
  0.03030681610107422,
  0.4493403434753418,
  0.04129528999328613,
  1.0774188041687012,
  0.10751700401306152,
  2.2989189624786377,
  0.21591544151306152,
  0.030628204345703125,
  0.42221760749816895,
  0.03804326057434082,
  1.0947840213775635,
  0.1236724853515625,
  2.285968542098999,
  0.23387622833251953,
  0.027799606323242188,
  0.47498631477355957,
  0.038399696350097656,
  1.190544605255127,
  0.11739659309387207,
  2.2424428462982178,
  0.2020246982574463,
  0.028720378875732422,
  0.4482462406158447,
  0.04457235336303711,
  1.0914018154144287,
  0.10721087455749512,
  2.233659267425537,
  0.22174692153930664,
  0.026659011840820312,
  0.42439770698547363,
  0.03803062438964844,
  1.1155691146850586,
  0.11873912811279297,
  3.931483507156372,
  0.21273493766784668,
  0.02734518051147461,
  0.42454051971435547,
  0.041069746017456055,
  1.0803730487823486,
  0.10291147232055664,
  2.2267627716064453,
  0.33333754539489746,
  0.

#### Execução do Método Guloso para Custo Beneficio

In [95]:
def execute_by_epoch_g_bc(epoch:int, data_op:list) -> tuple:
  start_time:float = 0.0
  time_execute:list = []
  predict_algorithm:list = []
  p_answer:pd.DataFrame = pd.DataFrame({})
  
  for _ in range(epoch):
    print("Epoch", _)
    for data in data_op:
      start_time = time()
      p_answer = by_benefit_cust(data[1], data[2], data[0])
      time_execute.append(
          time() - start_time
          )
      predict_algorithm.append(
          p_answer
      )

  return time_execute, predict_algorithm

In [96]:
pgbc:tuple = execute_by_epoch_g_bc(10, lista_data_file2)

Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9


In [99]:
pgbc[1].to

[   indice    value   weight
 0     0.0  10000.0  49877.0,    indice   value  weight
 0     0.0  1000.0  5002.0,    indice  value  weight
 0     0.0  100.0   995.0,    indice   value   weight
 0     0.0  2000.0  10011.0,    indice  value  weight
 0     0.0  200.0  1008.0,    indice   value   weight
 0     0.0  5000.0  25016.0,    indice  value  weight
 0     0.0  500.0  2543.0,    indice    value   weight
 0     0.0  10000.0  49877.0,    indice   value  weight
 0     0.0  1000.0  5002.0,    indice  value  weight
 0     0.0  100.0   995.0,    indice   value   weight
 0     0.0  2000.0  10011.0,    indice  value  weight
 0     0.0  200.0  1008.0,    indice   value   weight
 0     0.0  5000.0  25016.0,    indice  value  weight
 0     0.0  500.0  2543.0,    indice    value   weight
 0     0.0  10000.0  49519.0,    indice   value  weight
 0     0.0  1000.0  4990.0,    indice  value  weight
 0     0.0  100.0   997.0,    indice   value  weight
 0     0.0  2000.0  9819.0,    indice  value  wei

#### Execução do Método Dinamico

In [103]:
def execute_by_epoch_d(epoch:int, data_op:list) -> tuple:
  p_answer:int = 0
  start_time:float = 0.0
  time_execute:list = []
  predict_algorithm:pd.DataFrame = pd.DataFrame({})
  
  for _ in range(epoch):
    print("Epoch %d", _)
    for data in data_op:
      start_time = time()
      p_answer = knap_sack_memoize(data[0].shape[0], data[1]).dinamic_knap_sack(
          data[0].shape[0], 
          data[1], 
          data[0]
      )
      time_execute.append(
          time() - start_time
          )
      predict_algorithm.append(
          p_answer
      )
  
  return time_execute, predict_algorithm

In [102]:
lista_data_file2[0]

[      indice  value  weight  benefit_by_cust
 0          0  10000   49877              0.0
 5541    5541    347     638              0.0
 5540    5540    119     855              0.0
 5539    5539    192     418              0.0
 5538    5538    195     781              0.0
 ...      ...    ...     ...              ...
 1144    1144    641       1            641.0
 831      831    649       1            649.0
 2505    2505    651       1            651.0
 7069    7069    970       1            970.0
 8558    8558    978       1            978.0
 
 [10001 rows x 4 columns], 49877, 563647, array([0., 0., 0., ..., 0., 0., 0.])]

In [1]:
execute_by_epoch_d(10, lista_data_file2)

NameError: ignored

In [ ]:
def execute_by_epoch_dlru(epoch:int, data_op:list) -> tuple:
  p_answer:int = 0
  start_time:float = 0.0
  time_execute:list = []
  predict_algorithm:pd.DataFrame = pd.DataFrame({})
  
  for _ in range(epoch):
    print("Epoch %d", _)
    for data in data_op:
      start_time = time()
      p_answer = problem_knap_sack(
          data[0].shape[0], 
          data[1], 
          tuple(data[0].value.to_list()),
          tuple(data[0].weight.to_list())
      )
      time_execute.append(
          time() - start_time
          )
      predict_algorithm.append(
          p_answer
      )
  
  return time_execute, predict_algorithm